Verify that our `dmcg.py` script gets the same results on the drug_processed set as reported in the DMCG paper.

 * `drugs_processed/drugs.dmcg.sdf` - the test set smiles turned into sdf by dmcg.py
 * `drugs_processed/drugs.dmcg2.sdf` - same, generated with different random seed

 * `drugs_processed/drugs.ref.sdf.gz` - the reference sdfs

In [103]:
from rdkit import Chem
import gzip
import numpy as np
from tqdm import tqdm
from rdkit.Chem import rdMolAlign as MA
from rdkit.Chem import rdFMCS

from rdkit.Chem.rdmolops import RemoveHs
from rdkit import DataStructs
import multiprocessing
import copy

Adapting code from DMCG/evaluate.py

In [180]:
def get_best_rmsd(gen_mol, ref_mol):
    gen_mol = RemoveHs(gen_mol)
    ref_mol = RemoveHs(ref_mol)
    try:
        rmsd = MA.GetBestRMS(gen_mol, ref_mol)
    except:
        #lack perfect match
        mcs = rdFMCS.FindMCS([gen_mol,ref_mol],atomCompare=rdFMCS.AtomCompare.CompareAny,bondCompare=rdFMCS.BondCompare.CompareAny)
        submol = mcs.queryMol
        if mcs.numAtoms != gen_mol.GetNumAtoms():
            raise
        rms = []
        for a in gen_mol.GetSubstructMatches(submol):
            for b in ref_mol.GetSubstructMatches(submol):
                rms.append(MA.GetBestRMS(gen_mol,ref_mol,map = [list(zip(a,b))]))
        rmsd = np.min(rms)
        
    return rmsd

def get_rmsd_min(inputargs):
    mols, threshold = inputargs
    gen_mols, ref_mols = mols
    rmsd_mat = np.zeros([len(ref_mols), len(gen_mols)], dtype=np.float32)
    for i, gen_mol in enumerate(gen_mols):
        gen_mol_c = copy.deepcopy(gen_mol)
        for j, ref_mol in enumerate(ref_mols):
            ref_mol_c = copy.deepcopy(ref_mol)
            rmsd_mat[j, i] = get_best_rmsd(gen_mol_c, ref_mol_c)
    rmsd_mat_min = rmsd_mat.min(-1)
    return (rmsd_mat_min <= threshold).mean(), rmsd_mat_min.mean()


In [181]:
rmsd_mat.min(-1)

array([2.511996 , 2.431173 , 2.1458929, 2.3037882, 1.9481905, 2.4850197,
       2.4518783, 2.6659184, 2.3970592, 2.379571 , 2.4020786, 2.181613 ,
       1.7855121, 2.13329  , 1.6502929, 2.252782 , 2.257792 , 2.2047238,
       2.3032427, 2.2843485, 2.3084002, 1.8175057, 2.2844248, 1.9012657,
       2.320043 , 2.23418  , 2.657883 , 2.544342 , 2.5010183, 1.5962297,
       2.2741652, 2.1443455, 1.947057 , 2.4633627, 2.4573176, 2.6218529,
       1.8700916, 2.7246943, 1.7355028, 2.866373 , 2.948463 , 2.3839364,
       2.4039772, 1.7200216, 2.9209046, 1.2063236, 2.9300506, 2.9308527,
       1.9859898, 2.167662 , 2.9277284, 2.9930522, 2.2112591, 2.9588475,
       2.9564836, 3.0046372, 2.8061185, 2.9914172, 3.0071952, 2.9680064,
       3.0026522, 2.7887785, 2.560516 , 3.0236948, 2.9993813, 2.0033731,
       1.0226122, 2.3163717, 2.4694157, 1.5905163, 1.5856162, 2.5042882,
       2.2772048, 1.8546922, 1.761928 , 2.6115227, 2.6476068, 2.9476752,
       2.223573 , 2.614943 ], dtype=float32)

In [182]:
supp = Chem.ForwardSDMolSupplier(open('drugs_processed/drugs.dmcg.sdf','rb'))
mol_preds = [mol for mol in supp]

DMCG evaluate.py generates two conformers for each individual input, which seems suspect, but that's what they do (it adds a few percent of coverage).

In [183]:
supp = Chem.ForwardSDMolSupplier(open('drugs_processed/drugs.dmcg2.sdf','rb'))
mol_preds2 = [mol for mol in supp]

In [184]:
supp = Chem.ForwardSDMolSupplier(gzip.open('drugs_processed/drugs.ref.sdf.gz','rb'))
mol_labels = [mol for mol in supp]

In [185]:
smis = []
smimols = []
smifingers = []
for line in open('drugs_processed/drugs.smi'):
    m = Chem.MolFromSmiles(line.strip())
    smimols.append(m)
    smis.append(Chem.MolToSmiles(m,isomericSmiles=False))
    smifingers.append(Chem.RDKFingerprint(smimols[-1]))

In [186]:
smis[1363]

'C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1'

DMCG evaluate gets to replace coordinates on the same rdmol, so there's no issues matching up smiles, but when you go through sdf output and don't have explicit hydrogens this doesn't always work, so need to make sure our indices are lined up (they are).

In [187]:
smiles2pairs = dict()
for i,gen_mol in enumerate(mol_preds):
    smiles = Chem.MolToSmiles(gen_mol,isomericSmiles=False)
    if smiles != smis[i]:
        fp = Chem.RDKFingerprint(gen_mol)
        sim = DataStructs.FingerprintSimilarity(fp,smifingers[i])
        if sim != 1.0:
            print(i,sim,smiles,smis[i])
    smiles = smis[i]
    if smiles not in smiles2pairs:
        smiles2pairs[smiles] = [[gen_mol]]
    else:
        smiles2pairs[smiles][0].append(gen_mol)
    smiles2pairs[smiles][0].append(mol_preds2[i])
        
for i,ref_mol in enumerate(mol_labels):
    smiles = Chem.MolToSmiles(ref_mol,isomericSmiles=False)
    if smiles != smis[i]:
        fp = Chem.RDKFingerprint(gen_mol)
        sim = DataStructs.FingerprintSimilarity(fp,smifingers[i])
        if sim != 1.0:
            print(i,sim,smiles,smis[i])
    smiles = smis[i]
    
    if len(smiles2pairs[smiles]) == 1:
        smiles2pairs[smiles].append([ref_mol])
    else:
        smiles2pairs[smiles][1].append(ref_mol)


1363 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1364 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1365 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1366 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1367 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1368 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1369 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1370 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1371 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1372 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([

4051 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4052 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4053 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4054 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4055 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4056 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4057 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4058 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4059 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4060 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3

10007 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10008 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10009 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10010 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10011 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10012 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10013 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10014 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10015 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10016 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S

In [188]:
del_smiles = []
for smiles in smiles2pairs.keys():
    if len(smiles2pairs[smiles][1]) < 50 or len(smiles2pairs[smiles][1]) > 500:
        del_smiles.append(smiles)

In [189]:
len(del_smiles)

0

In [190]:
get_rmsd_min((smiles2pairs[smis[0]],1.25))

(0.975, 0.78643715)

In [191]:

cov_list = []
mat_list = []
pool = multiprocessing.Pool()

def input_args():
    for smiles in smiles2pairs.keys():
        yield smiles2pairs[smiles], 1.25

for res in tqdm(pool.imap(get_rmsd_min, input_args(), chunksize=10), total=len(smiles2pairs)):
    cov_list.append(res[0])
    mat_list.append(res[1])
print(f"cov mean {np.mean(cov_list)} med {np.median(cov_list)}")
print(f"mat mean {np.mean(mat_list)} med {np.median(mat_list)}")


100%|██████████| 200/200 [02:06<00:00,  1.58it/s] 

cov mean 0.9753586931752829 med 1.0
mat mean 0.6954147219657898 med 0.6825993657112122


Let's evaluate rdkit..

In [192]:
supp = Chem.ForwardSDMolSupplier(open('drugs_processed/drugs.rdkitm.sdf','rb'))
mol_preds = [mol for mol in supp]

In [196]:
supp = Chem.ForwardSDMolSupplier(open('drugs_processed/drugs.rdkit2.sdf','rb'))
mol_preds2 = [mol for mol in supp]

In [197]:
smiles2pairsrd = dict()
for i,gen_mol in enumerate(mol_preds):
    smiles = Chem.MolToSmiles(gen_mol,isomericSmiles=False)
    if smiles != smis[i]:
        fp = Chem.RDKFingerprint(gen_mol)
        sim = DataStructs.FingerprintSimilarity(fp,smifingers[i])
        if sim != 1.0:
            print(i,sim,smiles,smis[i])
    smiles = smis[i]
    if smiles not in smiles2pairsrd:
        smiles2pairsrd[smiles] = [[gen_mol]]
    else:
        smiles2pairsrd[smiles][0].append(gen_mol)
    smiles2pairsrd[smiles][0].append(mol_preds2[i])
        
for i,ref_mol in enumerate(mol_labels):
    smiles = Chem.MolToSmiles(ref_mol,isomericSmiles=False)
    if smiles != smis[i]:
        fp = Chem.RDKFingerprint(gen_mol)
        sim = DataStructs.FingerprintSimilarity(fp,smifingers[i])
        if sim != 1.0:
            print(i,sim,smiles,smis[i])
    smiles = smis[i]
    
    if len(smiles2pairsrd[smiles]) == 1:
        smiles2pairsrd[smiles].append([ref_mol])
    else:
        smiles2pairsrd[smiles][1].append(ref_mol)


1363 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1364 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1365 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1366 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1367 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1368 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1369 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1370 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1371 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([O-])[O-])N=N1 C=CCNc1nnc(Sc2ncccc2[N+](=O)[O-])s1
1372 0.23545210384959714 C=CCNC1=[S+]C(=[S+]c2ncccc2N([

4050 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4051 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4052 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4053 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4054 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4055 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4056 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4057 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4058 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3N=CC=CC3=C([O-])O)nc2n1 Cc1cc(C)n2cc(CSc3ncccc3C(=O)O)nc2n1
4059 0.24167378309137488 Cc1cc(C)n2cc(C[S+]=C3

10007 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10008 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10009 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10010 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10011 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10012 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10013 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10014 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10015 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S+]=C1c1ccccc1 Cc1cc(C(=O)NCCCn2ccnc2)sc1-c1ccccc1
10016 0.21643109540636044 CC1=CC(=C([O-])NCCCn2ccnc2)[S

In [198]:

cov_list = []
mat_list = []
pool = multiprocessing.Pool()

def input_args():
    for smiles in smiles2pairsrd.keys():
        yield smiles2pairsrd[smiles], 1.25

for res in tqdm(pool.imap(get_rmsd_min, input_args(), chunksize=10), total=len(smiles2pairs)):
    cov_list.append(res[0])
    mat_list.append(res[1])
print(f"cov mean {np.mean(cov_list)} med {np.median(cov_list)}")
print(f"mat mean {np.mean(mat_list)} med {np.median(mat_list)}")

100%|██████████| 200/200 [02:04<00:00,  1.61it/s] 

cov mean 0.6759354439155887 med 0.740281030444965
mat mean 1.107517123222351 med 0.9973100423812866
